In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def imputing_data_row(df,missing):
    df_a = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=-15)]
    df_b = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=15)]
    MOTOR = -1
    MOBIL = -1
    TRUK = -1
    if(len(df_a) + len(df_b) == 2):
        result = pd.concat([df_a,df_b])
        MOTOR = round(result['MOTOR'].mean())
        MOBIL = round(result['MOBIL'].mean())
        TRUK = round(result['TRUK/BUS'].mean())
    return MOTOR, MOBIL, TRUK 

def imputing_data(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]

    df_grouped = df_cleaned[['HARI', 'JAM', 'MOTOR', 'MOBIL','TRUK/BUS']]
    df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_grouped

def perhitungan_jumlah_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.2
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])
    df['JUMLAH'] = df['MOTOR'] + df['MOBIL'] + df['TRUK/BUS']
    return df

def menggabungkan_data(df, missing_times):
    # Menggabungkan data hilang dengan DataFrame utama
    df = pd.concat([df, missing_times], ignore_index=True)
    
    # Mengurutkan DataFrame berdasarkan kolom 'Waktu'
    df = df.sort_values(by=['TAHUN','BULAN','TANGGAL','HARI','JAM']).reset_index(drop=True)

    return df


In [3]:
path_1 = '../Data/Preprocessing/Data Losarang/2023/1_input_losarang_2023.xlsx'
path_2 = '../Data/Preprocessing/Data Losarang/2023/1_missing_times_losarang_2023.xlsx'

df = read_excel(path_1)
df_missing = read_excel(path_2)

df_missing.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-03 04:15:00,2023,1,3,Tuesday,04:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-03 05:15:00,2023,1,3,Tuesday,05:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-04 07:45:00,2023,1,4,Wednesday,07:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-05 00:00:00,2023,1,5,Thursday,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05 07:30:00,2023,1,5,Thursday,07:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for i, row in df_missing.iterrows():
    imputasi = imputing_data(df, row['HARI'], row['JAM'])
    break

In [5]:
imputasi

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS
0,Tuesday,04:15,43.0,53.0,75.0


In [6]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    MOTOR, MOBIL, TRUK = imputing_data_row(df, missing)
    if (MOTOR != -1) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = MOTOR
        df_missing.at[i, 'MOBIL'] = MOBIL
        df_missing.at[i, 'TRUK/BUS'] = TRUK

In [7]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    imputasi = imputing_data(df, missing['HARI'], missing['JAM'])
    if (not imputasi.empty) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = imputasi['MOTOR'].values[0]
        df_missing.at[i, 'MOBIL'] = imputasi['MOBIL'].values[0]
        df_missing.at[i, 'TRUK/BUS'] = imputasi['TRUK/BUS'].values[0]

In [8]:
df_missing = perhitungan_jumlah_smp(df_missing)

In [9]:
df_missing.to_excel('../Data/Preprocessing/Data Losarang/2023/2_missing_times_losarang_2023.xlsx', index=False)

In [10]:
df_missing.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-03 04:15:00,2023,1,3,Tuesday,04:15,20.0,34.0,22.0,76.0,4.0,34.0,55.0,93.0
1,2023-01-03 05:15:00,2023,1,3,Tuesday,05:15,42.0,36.0,28.0,106.0,8.4,36.0,70.0,114.0
2,2023-01-04 07:45:00,2023,1,4,Wednesday,07:45,178.0,78.0,82.0,338.0,35.6,78.0,205.0,319.0
3,2023-01-05 00:00:00,2023,1,5,Thursday,00:00,43.0,49.0,40.0,132.0,8.6,49.0,100.0,158.0
4,2023-01-05 07:30:00,2023,1,5,Thursday,07:30,182.0,74.0,104.0,360.0,36.4,74.0,260.0,370.0


In [11]:
df_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14750 entries, 0 to 14749
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   WAKTU     14750 non-null  datetime64[ns]
 1   TAHUN     14750 non-null  int64         
 2   BULAN     14750 non-null  int64         
 3   TANGGAL   14750 non-null  int64         
 4   HARI      14750 non-null  object        
 5   JAM       14750 non-null  object        
 6   MOTOR     14750 non-null  float64       
 7   MOBIL     14750 non-null  float64       
 8   TRUK/BUS  14750 non-null  float64       
 9   JUMLAH    14750 non-null  float64       
 10  SMP_MTR   14750 non-null  float64       
 11  SMP_MBL   14750 non-null  float64       
 12  SMP_TRK   14750 non-null  float64       
 13  SMP       14750 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(3), object(2)
memory usage: 1.6+ MB


In [12]:
df_compiled = menggabungkan_data(df, df_missing)
df_compiled.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,Sunday,00:00,85.0,24.0,4.0,113.0,17.0,24.0,10.0,51.0
1,2023-01-01 00:15:00,2023,1,1,Sunday,00:15,92.0,19.0,6.0,117.0,18.4,19.0,15.0,52.0
2,2023-01-01 00:30:00,2023,1,1,Sunday,00:30,78.0,16.0,6.0,100.0,15.6,16.0,15.0,47.0
3,2023-01-01 00:45:00,2023,1,1,Sunday,00:45,78.0,23.0,4.0,105.0,15.6,23.0,10.0,49.0
4,2023-01-01 01:00:00,2023,1,1,Sunday,01:00,76.0,14.0,6.0,96.0,15.2,14.0,15.0,44.0


In [13]:
df_compiled.to_excel('../Data/Preprocessing/Data Losarang/2023/2_input_losarang_2023.xlsx', index=False)